In [62]:
import requests
import pandas as pd

# Global Request Functions

In [63]:
header = {"X-Riot-Token": "RGAPI-7e521a6d-79b3-4b7c-bfad-3c6adcd287c2"}


def getSummoner(summoner):
    summoner = requests.get(
        f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summoner}",
        headers=header,
    )
    return summoner.json()


def getMatchIDs(puuid):
    matchIDs = requests.get(
        f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=100",
        headers=header,
    )
    return matchIDs.json()


def getMatch(matchID):
    matches = requests.get(
        f"https://americas.api.riotgames.com/lol/match/v5/matches/{matchID}",
        headers=header,
    )
    return matches.json()

In [64]:
champions = {}

In [65]:

class Summoner:
    def __init__(self, summonerName):
        self.name = summonerName
        self.puuid = getSummoner(summonerName)["puuid"]
        self.matchIDs = getMatchIDs(self.puuid)
        self.partnerName = ""
        self.partnerPuuid = ""
        self.df = pd.DataFrame(columns=["MatchID", "ChampionID", "VisionScore", "KDA",
                           "PartnerChampionID", "PartnerVisionScore", "PartnerKDA", "Win/Lost"])

    def initPartner(self, partnerName):
        r = getSummoner(partnerName)
        self.partnerPuuid = r["puuid"]
        self.partnerName = r["name"]

        for i, matchID in enumerate(self.matchIDs):
            matchJSON = getMatch(matchID)


            try:
                if matchJSON["info"]["gameMode"] == "ARAM":
                    continue
 
                participants = matchJSON["info"]["participants"]
            except:
                break

            if self.partnerPuuid in matchJSON["metadata"]["participants"]:
                myNdx = 0
                partnerNdx = 0

                while participants[myNdx]["summonerName"] != self.name:
                    myNdx += 1
                while participants[partnerNdx]["summonerName"] != self.partnerName:
                    partnerNdx += 1

                me = participants[myNdx]
                partner = participants[partnerNdx]

                kda = round(participants[myNdx]["challenges"]["kda"], 2)
                partnerKDA = round(participants[partnerNdx]["challenges"]["kda"],  2)

                visionScore = round(participants[myNdx]["visionScore"], 2)
                partnerVisionScore = round(participants[partnerNdx]["visionScore"], 2)

                myChampion = me["championName"]
                partnersChampion = partner["championName"]

                if champions.get(myChampion) == None:
                    champions[myChampion] = me["championId"]
                if champions.get(myChampion) == None:
                    champions[myChampion] = partner["championId"]

                if me["win"]:
                    win = 1
                else:
                    win = 0


                newMatch = pd.Series(
                    [
                        matchID, myChampion, visionScore ,kda, partnersChampion, partnerVisionScore, partnerKDA, win
                    ],
                    index=["MatchID", "ChampionID", "VisionScore", "KDA",
                           "PartnerChampionID", "PartnerVisionScore", "PartnerKDA", "Win/Lost"],
                )

                print(f"Adding: {[matchID, myChampion, visionScore, kda, partnersChampion, partnerVisionScore, partnerKDA, win]} to DF")
                self.df.loc[i] = newMatch

In [66]:
mainSummoner = Summoner("41619")
mainSummoner.initPartner("yongbin")

In [67]:
mainSummoner.df.to_csv('41binMatches.csv')


In [68]:
print(champions)

{'Rakan': 497, 'Pyke': 555, 'Alistar': 12, 'Renata': 888, 'Xerath': 101, 'Thresh': 412, 'Morgana': 25, 'Senna': 235, 'Lux': 99, 'Bard': 432, 'Velkoz': 161, 'Nautilus': 111, 'Fiora': 114, 'Kled': 240, 'Ekko': 245, 'Zeri': 221, 'Jinx': 222, 'Vayne': 67, 'Yasuo': 157, 'Draven': 119, 'Zilean': 26, 'Yuumi': 350, 'Shen': 98, 'Rell': 526, 'LeeSin': 64, 'Braum': 201, 'Blitzcrank': 53}
